### **The code is take from the following project and github repository.**
[https://github.com/almarengo/gpt2-text-classification/tree/main
](https://github.com/almarengo/gpt2-text-classification/tree/main)

In [ ]:
!pip install transformers
!pip install torch
!pip install pynvml
!pip install psutil
!pip install Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.6 MB/s eta 0:00:00


In [ ]:
import math
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2Model
from google.colab import drive
import pandas as pd
import subprocess
import time
import psutil
import threading
import csv
import os

CARBON_INTENSITY = 0.4
RESULTS_CSV = "energy_measurements_gpu.csv"
USE_GPU = tf.config.list_physical_devices('GPU')


def get_gpu_power():
    if USE_GPU:
        result = subprocess.run(
            ["nvidia-smi", "--query-gpu=power.draw", "--format=csv,nounits,noheader"],
            stdout=subprocess.PIPE, text=True)
        return float(result.stdout.strip())
    return 0.0

def get_cpu_power_estimate():
    cpu_util = psutil.cpu_percent()
    estimated_cpu_power = (cpu_util / 100.0) * 50
    return estimated_cpu_power

def measure_energy_and_time(func, *args, **kwargs):
    start_time = time.time()
    energy_gpu_joules = 0.0
    energy_cpu_joules = 0.0
    interval = 0.1

    def target():
        func(*args, **kwargs)

    thread = threading.Thread(target=target)
    thread.start()

    while thread.is_alive():
        gpu_watts = get_gpu_power()
        cpu_watts = get_cpu_power_estimate()
        time.sleep(interval)
        energy_gpu_joules += gpu_watts * interval
        energy_cpu_joules += cpu_watts * interval

    thread.join()
    end_time = time.time()
    total_time = end_time - start_time
    total_energy_joules = energy_gpu_joules + energy_cpu_joules
    total_energy_kwh = total_energy_joules * 2.77778e-7
    emissions_kg = total_energy_kwh * CARBON_INTENSITY

    return total_time, total_energy_joules, emissions_kg

def log_results(task_name, phase, total_time, energy_joules, emissions_kg, filename=RESULTS_CSV):
    file_exists = os.path.isfile(filename)
    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(["Task", "Phase", "Time(s)", "Energy(J)", "Emissions(kgCO2)"])
        writer.writerow([task_name, phase, total_time, energy_joules, emissions_kg])


df = pd.read_csv('tweets_processed.csv')

df = df.sample(frac=0.1, random_state=42).reset_index(drop=True)

from sklearn.model_selection import train_test_split

X = df['lemmatized_tweets']
y = df['VADER_sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)


MAX_LENGTH = math.ceil((X_train.apply(lambda x: len(str(x).split())).mean()))+2

PAD_TOKEN = "<|pad|>"
EOS_TOKEN = "<|endoftext|>"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2",
    pad_token=PAD_TOKEN,
    eos_token=EOS_TOKEN,
    max_length=MAX_LENGTH,
    is_split_into_words=True)

X_train = [str(ex) + EOS_TOKEN for ex in X_train]
X_test = [str(ex) + EOS_TOKEN for ex in X_test]

X_train_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
                      truncation=True, padding="max_length", add_special_tokens=True)['input_ids'] for x in X_train]
X_test_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
                     truncation=True, padding="max_length", add_special_tokens=True)['input_ids'] for x in X_test]

X_train_in = tf.squeeze(tf.convert_to_tensor(X_train_), axis=1)
X_test_in = tf.squeeze(tf.convert_to_tensor(X_test_), axis=1)

X_train_mask_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
                           truncation=True, padding="max_length", add_special_tokens=True)["attention_mask"] for x in X_train]
X_test_mask_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
                          truncation=True, padding="max_length", add_special_tokens=True)["attention_mask"] for x in X_test]

X_train_mask = tf.squeeze(tf.convert_to_tensor(X_train_mask_), axis=1)
X_test_mask = tf.squeeze(tf.convert_to_tensor(X_test_mask_), axis=1)


model = TFGPT2Model.from_pretrained("gpt2", use_cache=False,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id)
model.training = True
model.resize_token_embeddings(len(tokenizer))

for layer in model.layers:
    layer.trainable = False

input_ = tf.keras.layers.Input(shape=(None,), dtype='int32')
mask_ = tf.keras.layers.Input(shape=(None,), dtype='int32')
x = model(input_, attention_mask=mask_)
x = tf.reduce_mean(x.last_hidden_state, axis=1)
x = tf.keras.layers.Dense(16, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(3, activation='softmax')(x)

clf = tf.keras.Model([input_, mask_], output)

base_learning_rate = 0.0005
optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate)
loss = tf.keras.losses.SparseCategoricalCrossentropy()

clf.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

callbacks = tf.keras.callbacks.EarlyStopping(
    monitor="accuracy", verbose=1, patience=3, restore_best_weights=True)

def map_sentiment(value):
  if value == 'Negative':
    return 0
  if value == 'Neutral':
    return 1
  if value == 'Positive':
    return 2

y_train_ = y_train.map(map_sentiment)
y_test_ = y_test.map(map_sentiment)

y_train_in = tf.constant(y_train_, dtype=tf.int32)
y_test_in = tf.constant(y_test_, dtype=tf.int32)

tf.config.experimental_run_functions_eagerly(True)

def train_model():
    clf.fit([X_train_in, X_train_mask], y_train_in, epochs=5, batch_size=32, validation_split=0.2, callbacks=callbacks)

print("Starting training...")
train_time, train_energy_j, train_emissions_kg = measure_energy_and_time(train_model)
print(f"Training completed in {train_time:.2f}s, Energy: {train_energy_j:.2f}J, Emissions: {train_emissions_kg:.4f}kg CO2")
log_results("GPT2_TransferLearning", "Training", train_time, train_energy_j, train_emissions_kg)


print("Evaluating on test set...")
eval_time, eval_energy_j, eval_emissions_kg = measure_energy_and_time(clf.evaluate, [X_test_in, X_test_mask], y_test_in)
print(f"Evaluation completed in {eval_time:.2f}s, Energy: {eval_energy_j:.2f}J, Emissions: {eval_emissions_kg:.4f}kg CO2")
log_results("GPT2_TransferLearning", "Evaluation", eval_time, eval_energy_j, eval_emissions_kg)

clf.training = False
y_pred = clf.predict([X_test_in, X_test_mask])

print("Done. Results saved to:", RESULTS_CSV)


All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


Starting training...
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


4/4 [==============================] - ETA: 0s - loss: 6.8715 - accuracy: 0.2984

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


4/4 [==============================] - 4s 993ms/step - loss: 6.8715 - accuracy: 0.2984 - val_loss: 8.1579 - val_accuracy: 0.1250
Epoch 2/5
4/4 [==============================] - 3s 775ms/step - loss: 6.4132 - accuracy: 0.3065 - val_loss: 6.4392 - val_accuracy: 0.1250
Epoch 3/5
4/4 [==============================] - 4s 983ms/step - loss: 5.2451 - accuracy: 0.3145 - val_loss: 4.8973 - val_accuracy: 0.1250
Epoch 4/5
4/4 [==============================] - 3s 750ms/step - loss: 4.5685 - accuracy: 0.3468 - val_loss: 3.6648 - val_accuracy: 0.1250
Epoch 5/5
4/4 [==============================] - 2s 624ms/step - loss: 3.8470 - accuracy: 0.3468 - val_loss: 2.8701 - val_accuracy: 0.2500
Restoring model weights from the end of the best epoch: 4.
Training completed in 20.58s, Energy: 1066.56J, Emissions: 0.0001kg CO2
Evaluating on test set...


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


2/2 [==============================] - 1s 344ms/step - loss: 3.0356 - accuracy: 0.2830
Evaluation completed in 0.81s, Energy: 49.05J, Emissions: 0.0000kg CO2


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


2/2 [==============================] - 0s 213ms/step
Done. Results saved to: energy_measurements.csv


### Remove Frezzing

In [ ]:
import math
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2Model
from google.colab import drive
import pandas as pd
import subprocess
import time
import psutil
import threading
import csv
import os
import pynvml


CARBON_INTENSITY = 0.4
RESULTS_CSV = "energy_measurements_freze_gpu.csv"
GPU_DETAILS_CSV = "gpu_details_freze_gpu.csv"
USE_GPU = tf.config.list_physical_devices('GPU')


def get_gpu_details():
    pynvml.nvmlInit()
    device_count = pynvml.nvmlDeviceGetCount()
    details = []

    for i in range(device_count):
        handle = pynvml.nvmlDeviceGetHandleByIndex(i)
        name = pynvml.nvmlDeviceGetName(handle)  # No need to decode
        memory_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        utilization = pynvml.nvmlDeviceGetUtilizationRates(handle)
        architecture = pynvml.nvmlSystemGetCudaDriverVersion()

        details.append({
            "GPU": name,
            "Memory Total (MB)": memory_info.total // (1024 ** 2),
            "Memory Free (MB)": memory_info.free // (1024 ** 2),
            "Memory Used (MB)": memory_info.used // (1024 ** 2),
            "GPU Utilization (%)": utilization.gpu,
            "Memory Utilization (%)": utilization.memory,
            "CUDA Version": architecture
        })

    pynvml.nvmlShutdown()
    return details


def get_gpu_power():
    if USE_GPU:
        result = subprocess.run(
            ["nvidia-smi", "--query-gpu=power.draw", "--format=csv,nounits,noheader"],
            stdout=subprocess.PIPE, text=True)
        return float(result.stdout.strip())
    return 0.0

def get_cpu_power_estimate():
    cpu_util = psutil.cpu_percent()
    estimated_cpu_power = (cpu_util / 100.0) * 50
    return estimated_cpu_power

def measure_energy_and_time(func, *args, **kwargs):
    start_time = time.time()
    energy_gpu_joules = 0.0
    energy_cpu_joules = 0.0
    interval = 0.1

    def target():
        func(*args, **kwargs)

    thread = threading.Thread(target=target)
    thread.start()

    while thread.is_alive():
        gpu_watts = get_gpu_power()
        cpu_watts = get_cpu_power_estimate()
        time.sleep(interval)
        energy_gpu_joules += gpu_watts * interval
        energy_cpu_joules += cpu_watts * interval

    thread.join()
    end_time = time.time()
    total_time = end_time - start_time
    total_energy_joules = energy_gpu_joules + energy_cpu_joules
    total_energy_kwh = total_energy_joules * 2.77778e-7
    emissions_kg = total_energy_kwh * CARBON_INTENSITY

    return total_time, total_energy_joules, emissions_kg

def log_results(task_name, phase, total_time, energy_joules, emissions_kg, filename=RESULTS_CSV):
    file_exists = os.path.isfile(filename)
    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(["Task", "Phase", "Time(s)", "Energy(J)", "Emissions(kgCO2)"])
        writer.writerow([task_name, phase, total_time, energy_joules, emissions_kg])

def log_gpu_details(details, filename=GPU_DETAILS_CSV):
    file_exists = os.path.isfile(filename)
    with open(filename, 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=details[0].keys())
        if not file_exists:
            writer.writeheader()
        writer.writerows(details)


df = pd.read_csv('tweets_processed.csv')

df = df.sample(frac=0.6, random_state=42).reset_index(drop=True)

from sklearn.model_selection import train_test_split

X = df['lemmatized_tweets']
y = df['VADER_sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)


MAX_LENGTH = math.ceil((X_train.apply(lambda x: len(str(x).split())).mean()))+2

PAD_TOKEN = "<|pad|>"
EOS_TOKEN = "<|endoftext|>"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2",
    pad_token=PAD_TOKEN,
    eos_token=EOS_TOKEN,
    max_length=MAX_LENGTH,
    is_split_into_words=True)

X_train = [str(ex) + EOS_TOKEN for ex in X_train]
X_test = [str(ex) + EOS_TOKEN for ex in X_test]

X_train_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
                      truncation=True, padding="max_length", add_special_tokens=True)['input_ids'] for x in X_train]
X_test_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
                     truncation=True, padding="max_length", add_special_tokens=True)['input_ids'] for x in X_test]

X_train_in = tf.squeeze(tf.convert_to_tensor(X_train_), axis=1)
X_test_in = tf.squeeze(tf.convert_to_tensor(X_test_), axis=1)

X_train_mask_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
                           truncation=True, padding="max_length", add_special_tokens=True)["attention_mask"] for x in X_train]
X_test_mask_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH,
                          truncation=True, padding="max_length", add_special_tokens=True)["attention_mask"] for x in X_test]

X_train_mask = tf.squeeze(tf.convert_to_tensor(X_train_mask_), axis=1)
X_test_mask = tf.squeeze(tf.convert_to_tensor(X_test_mask_), axis=1)


model = TFGPT2Model.from_pretrained("gpt2", use_cache=False,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id)
model.training = True
model.resize_token_embeddings(len(tokenizer))

input_ = tf.keras.layers.Input(shape=(None,), dtype='int32')
mask_ = tf.keras.layers.Input(shape=(None,), dtype='int32')
x = model(input_, attention_mask=mask_)
x = tf.reduce_mean(x.last_hidden_state, axis=1)
x = tf.keras.layers.Dense(16, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(3, activation='softmax')(x)

clf = tf.keras.Model([input_, mask_], output)

base_learning_rate = 0.0005
optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate)
loss = tf.keras.losses.SparseCategoricalCrossentropy()

clf.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

callbacks = tf.keras.callbacks.EarlyStopping(
    monitor="accuracy", verbose=1, patience=3, restore_best_weights=True)

def map_sentiment(value):
  if value == 'Negative':
    return 0
  if value == 'Neutral':
    return 1
  if value == 'Positive':
    return 2

y_train_ = y_train.map(map_sentiment)
y_test_ = y_test.map(map_sentiment)

y_train_in = tf.constant(y_train_, dtype=tf.int32)
y_test_in = tf.constant(y_test_, dtype=tf.int32)

tf.config.experimental_run_functions_eagerly(True)

def train_model():
    clf.fit([X_train_in, X_train_mask], y_train_in, epochs=30, batch_size=32, validation_split=0.2, callbacks=callbacks)


gpu_details = get_gpu_details()
print("GPU Details and Insights:")
for detail in gpu_details:
    print(detail)
log_gpu_details(gpu_details)


print("Starting training...")
train_time, train_energy_j, train_emissions_kg = measure_energy_and_time(train_model)
print(f"Training completed in {train_time:.2f}s, Energy: {train_energy_j:.2f}J, Emissions: {train_emissions_kg:.4f}kg CO2")
log_results("GPT2_TransferLearning", "Training", train_time, train_energy_j, train_emissions_kg)

print("Evaluating on test set...")
eval_time, eval_energy_j, eval_emissions_kg = measure_energy_and_time(clf.evaluate, [X_test_in, X_test_mask], y_test_in)
print(f"Evaluation completed in {eval_time:.2f}s, Energy: {eval_energy_j:.2f}J, Emissions: {eval_emissions_kg:.4f}kg CO2")
log_results("GPT2_TransferLearning", "Evaluation", eval_time, eval_energy_j, eval_emissions_kg)

clf.training = False
y_pred = clf.predict([X_test_in, X_test_mask])

print("Done. Results saved to:", RESULTS_CSV)


All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


GPU Details and Insights:
{'GPU': 'Tesla T4', 'Memory Total (MB)': 15360, 'Memory Free (MB)': 6793, 'Memory Used (MB)': 8566, 'GPU Utilization (%)': 0, 'Memory Utilization (%)': 0, 'CUDA Version': 12020}
Starting training...


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/30
24/24 [==============================] - ETA: 0s - loss: 1.8019 - accuracy: 0.3533

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


24/24 [==============================] - 27s 1s/step - loss: 1.8019 - accuracy: 0.3533 - val_loss: 1.0890 - val_accuracy: 0.4392
Epoch 2/30
24/24 [==============================] - 25s 1s/step - loss: 1.1001 - accuracy: 0.3732 - val_loss: 1.0596 - val_accuracy: 0.4603
Epoch 3/30
24/24 [==============================] - 25s 1s/step - loss: 1.0729 - accuracy: 0.4170 - val_loss: 1.0744 - val_accuracy: 0.3228
Epoch 4/30
24/24 [==============================] - 25s 1s/step - loss: 1.0762 - accuracy: 0.3559 - val_loss: 1.0937 - val_accuracy: 0.3228
Epoch 5/30
24/24 [==============================] - 25s 1s/step - loss: 1.0842 - accuracy: 0.3679 - val_loss: 1.0790 - val_accuracy: 0.3228
Epoch 6/30
24/24 [==============================] - 26s 1s/step - loss: 1.0688 - accuracy: 0.3665 - val_loss: 1.0775 - val_accuracy: 0.4603
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 3.
Training completed in 154.58s, Energy: 9198.47J, Emissions: 0.0010kg CO2
Evaluating on t

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


10/10 [==============================] - 2s 224ms/step - loss: 1.0654 - accuracy: 0.3599
Evaluation completed in 50.29s, Energy: 1819.57J, Emissions: 0.0002kg CO2


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


10/10 [==============================] - 2s 200ms/step
Done. Results saved to: energy_measurements_freze_gpu.csv


In [ ]:
!nvidia-smi

Tue Dec 10 13:23:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--